In [3]:
import pandas as pd

# Utils Functions

# 1. Data Loading

# 2. Initial Data Quality Assessment

# 3. Numeric Data Consistency

# 4. Data Integrity and Duplicates

# 5. Data Enrichment

# 6. Data Transformation and Standardization

# 7. Pipeline Design

# 8. Bonus Challenge Question - Actor Collaboration and Performance Analysis